# 🎯 RoastFormer Evaluation & Demo

**Comprehensive evaluation and interactive demonstration**

Author: Charlee Kraiss  
Project: RoastFormer - Transformer-Based Roast Profile Generation  
Date: November 2024

---

## 📋 What This Notebook Does

1. ✅ Loads best trained model from training experiments
2. ✅ Evaluates on validation set with comprehensive metrics
3. ✅ Generates sample profiles (real vs generated comparisons)
4. ✅ Computes evaluation metrics (MAE, DTW, Physics, Finish Temp)
5. ✅ Creates beautiful visualizations
6. ✅ Interactive demo (generate custom profiles)
7. ✅ Packages results for presentation

**Perfect for:** Live demo during capstone presentation!

**Estimated Runtime:** 30-60 minutes

---

## 🎯 Prerequisites

Before running this notebook:
1. ✅ Complete training (run `RoastFormer_Training_Suite.ipynb`)
2. ✅ Download results package
3. ✅ Extract and identify best model checkpoint
4. ✅ Upload checkpoint to Google Drive

---

## 1️⃣ Setup Environment

In [2]:
# Check GPU availability (optional for evaluation, but nice to have)
import torch
print("="*80)
print("ENVIRONMENT CHECK")
print("="*80)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = 'cuda'
else:
    print("Running on CPU (okay for evaluation)")
    device = 'cpu'
print("="*80)

ENVIRONMENT CHECK
CUDA available: True
GPU: Tesla T4


In [3]:
# Install required packages
!pip install -q pandas scikit-learn matplotlib seaborn numpy

print("✅ Dependencies installed")

✅ Dependencies installed


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Navigate to project directory
%cd /content/gdrive/MyDrive/"Colab Notebooks"/"GEN_AI"

/content/gdrive/MyDrive/Colab Notebooks/GEN_AI


In [7]:
# Extract data (same as training notebook)
import zipfile
import os

print("="*80)
print("EXTRACTING DATA")
print("="*80)

zip_path = '/content/gdrive/MyDrive/Colab Notebooks/GEN_AI/roastformer_data_20251118_090504.zip'

if os.path.exists(zip_path):
    os.chdir('/content')
    print(f"Working directory: {os.getcwd()}")

    print(f"\n📦 Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')

    print("✅ Extraction complete")

    import json
    with open('preprocessed_data/dataset_stats.json', 'r') as f:
        stats = json.load(f)
    print(f"\n📊 Dataset: {stats['total_profiles']} profiles")
else:
    print(f"❌ Zip not found at: {zip_path}")

print("="*80)

EXTRACTING DATA
Working directory: /content

📦 Extracting...
✅ Extraction complete

📊 Dataset: 144 profiles


## 2️⃣ Load Best Model

**👉 UPDATE THIS PATH 👈**

After training, you'll know which model performed best. Update the path below:

In [8]:
# ═══════════════════════════════════════════════════════════════
# CONFIGURE CHECKPOINT PATH
# ═══════════════════════════════════════════════════════════════

# TODO: Update this path after training
# Example: If baseline_sinusoidal was best, use that checkpoint
CHECKPOINT_PATH = '/content/gdrive/MyDrive/Colab Notebooks/GEN_AI/roastformer_ALL_EXPERIMENTS_20251118_151724/checkpoints/baseline_sinusoidal_model.pt'

# OR if you uploaded directly:
# CHECKPOINT_PATH = '/content/gdrive/MyDrive/roastformer_best_model.pt'

print(f"Checkpoint path: {CHECKPOINT_PATH}")
print(f"Exists: {os.path.exists(CHECKPOINT_PATH)}")

Checkpoint path: /content/gdrive/MyDrive/Colab Notebooks/GEN_AI/roastformer_ALL_EXPERIMENTS_20251118_151724/checkpoints/baseline_sinusoidal_model.pt
Exists: True


In [9]:
# Load checkpoint
import sys
sys.path.append('.')

print("="*80)
print("LOADING MODEL CHECKPOINT")
print("="*80)

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

print(f"\n✅ Checkpoint loaded successfully")
print(f"\nModel Info:")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Best Val Loss: {checkpoint['best_val_loss']:.4f}°F")
print(f"  Configuration:")
for key, value in checkpoint['config'].items():
    if key not in ['device', 'checkpoint_dir', 'results_dir', 'preprocessed_dir']:
        print(f"    {key}: {value}")

print("="*80)

LOADING MODEL CHECKPOINT

✅ Checkpoint loaded successfully

Model Info:
  Epoch: 16
  Best Val Loss: 70947.5547°F
  Configuration:
    d_model: 256
    nhead: 8
    num_layers: 6
    dim_feedforward: 1024
    embed_dim: 32
    dropout: 0.1
    batch_size: 8
    num_epochs: 100
    learning_rate: 0.0001
    weight_decay: 0.01
    grad_clip: 1.0
    early_stopping_patience: 15
    max_sequence_length: 800
    save_every: 10
    positional_encoding: sinusoidal
    experiment_name: baseline_sinusoidal


In [12]:
# Initialize model from checkpoint
from src.model.transformer_adapter import AdaptedConditioningModule, AdaptedRoastFormer
from src.dataset.preprocessed_data_loader import PreprocessedDataLoader

print("="*80)
print("INITIALIZING MODEL FROM CHECKPOINT")
print("="*80)

# Load data to get feature dimensions
data_loader = PreprocessedDataLoader(preprocessed_dir='preprocessed_data')
train_profiles, val_profiles = data_loader.load_data()

# Get feature dimensions from data loader
feature_dims = data_loader.get_feature_dimensions()

print(f"\n📊 Feature Dimensions:")
print(f"   Origins: {feature_dims['num_origins']}")
print(f"   Processes: {feature_dims['num_processes']}")
print(f"   Roast Levels: {feature_dims['num_roast_levels']}")
print(f"   Varieties: {feature_dims['num_varieties']}")
print(f"   Flavors: {feature_dims['num_flavors']}")

# Get model config from checkpoint
config = checkpoint['config']

# Initialize conditioning module
conditioning_module = AdaptedConditioningModule(
    num_origins=feature_dims['num_origins'],
    num_processes=feature_dims['num_processes'],
    num_roast_levels=feature_dims['num_roast_levels'],
    num_varieties=feature_dims['num_varieties'],
    num_flavors=feature_dims['num_flavors'],
    embed_dim=config['embed_dim']
)

print(f"\n✅ Conditioning module initialized")

# Initialize model
model = AdaptedRoastFormer(
    conditioning_module=conditioning_module,
    d_model=config['d_model'],
    nhead=config['nhead'],
    num_layers=config['num_layers'],
    dim_feedforward=config['dim_feedforward'],
    dropout=config['dropout'],
    positional_encoding=config['positional_encoding'],
    max_seq_len=config['max_sequence_length']
)

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print(f"✅ Model loaded: {sum(p.numel() for p in
model.parameters()):,} parameters")
print("="*80)

INITIALIZING MODEL FROM CHECKPOINT

LOADING PREPROCESSED DATA
✓ Loaded 123 training profiles
✓ Loaded 21 validation profiles
✓ Loaded metadata

📊 Feature Vocabulary:
   Origins: 19
   Processes: 13
   Roast Levels: 7
   Varieties: 25
   Flavors: 98


📊 Feature Dimensions:
   Origins: 19
   Processes: 13
   Roast Levels: 7
   Varieties: 25
   Flavors: 98

✅ Conditioning module initialized
✅ Model loaded: 6,376,673 parameters


## 3️⃣ Validation Set Evaluation

Generate profiles for all validation samples and compute metrics.

In [ ]:
# Generate profiles for all validation samples
import numpy as np
from tqdm import tqdm

print("="*80)
print(f"GENERATING PROFILES FOR {len(val_profiles)} VALIDATION SAMPLES")
print("="*80)

generated_profiles = []
real_profiles = []

with torch.no_grad():
    for idx in tqdm(range(len(val_profiles))):
        # Get real profile
        real_profile = val_profiles[idx]
        metadata = val_metadata.iloc[idx]

        # Prepare conditioning (simplified - adapt based on your data loader)
        # TODO: Use actual encoding from data loader
        # For now, placeholder

        # Generate profile
        # TODO: Implement generation loop
        # generated = model.generate(conditioning, start_temp, max_steps)

        # Store results
        real_profiles.append(real_profile)
        # generated_profiles.append(generated)

print(f"\n✅ Generated {len(generated_profiles)} profiles")
print("⚠️  Note: Full generation implementation needed (see evaluate_transformer.py)")

## 4️⃣ Compute Evaluation Metrics

**Metrics:**
1. **MAE (Mean Absolute Error)** - Average temperature difference
2. **DTW (Dynamic Time Warping)** - Shape similarity
3. **Finish Temperature Accuracy** - Hit target roast level
4. **Physics Compliance** - Monotonicity, bounded RoR

In [ ]:
# Compute metrics (placeholder - implement with actual generated profiles)
print("="*80)
print("EVALUATION METRICS")
print("="*80)

# Placeholder metrics
metrics = {
    'mae': 0.0,  # TODO: Compute actual MAE
    'dtw': 0.0,  # TODO: Compute actual DTW
    'finish_temp_accuracy': 0.0,  # TODO: Compute percentage within 10°F
    'physics_compliance': {
        'monotonicity': 0.0,  # TODO: Check post-turning-point monotonicity
        'bounded_ror': 0.0,   # TODO: Check 20-100°F/min RoR
        'smooth_transitions': 0.0  # TODO: Check <10°F jumps
    }
}

print(f"\nMetrics (to be computed):")
print(f"  MAE: {metrics['mae']:.2f}°F")
print(f"  DTW Distance: {metrics['dtw']:.2f}")
print(f"  Finish Temp Accuracy: {metrics['finish_temp_accuracy']:.1f}%")
print(f"  Physics Compliance:")
print(f"    Monotonicity: {metrics['physics_compliance']['monotonicity']:.1f}%")
print(f"    Bounded RoR: {metrics['physics_compliance']['bounded_ror']:.1f}%")
print(f"    Smooth Transitions: {metrics['physics_compliance']['smooth_transitions']:.1f}%")

print("\n⚠️  Full metric computation to be implemented")
print("   See evaluate_transformer.py for reference implementation")
print("="*80)

## 5️⃣ Visual Comparisons (Real vs Generated)

Create beautiful side-by-side plots for presentation.

In [ ]:
# Plot real vs generated profiles
import matplotlib.pyplot as plt

# Placeholder visualization
print("Creating visualizations...")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Real vs Generated Roast Profiles', fontsize=16, fontweight='bold')

# Plot 6 examples
for idx in range(6):
    ax = axes[idx // 3, idx % 3]

    # TODO: Plot actual profiles
    # ax.plot(time, real_profile, label='Real', linewidth=2, color='blue')
    # ax.plot(time, generated_profile, label='Generated', linewidth=2, color='red', linestyle='--')

    # Placeholder
    ax.text(0.5, 0.5, f'Example {idx+1}\n(To be plotted)',
            ha='center', va='center', fontsize=12, transform=ax.transAxes)

    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Temperature (°F)')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
os.makedirs('evaluation_results', exist_ok=True)
plt.savefig('evaluation_results/real_vs_generated_profiles.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Saved: evaluation_results/real_vs_generated_profiles.png")
print("⚠️  Update with actual profile data")

## 6️⃣ Interactive Demo (Custom Profile Generation)

**Perfect for live presentation demo!**

Generate a custom profile by specifying bean characteristics and flavors.

In [ ]:
# ═══════════════════════════════════════════════════════════════
# INTERACTIVE DEMO
# ═══════════════════════════════════════════════════════════════

print("="*80)
print("CUSTOM PROFILE GENERATION DEMO")
print("="*80)

# Define demo inputs
demo_specs = {
    'origin': 'Ethiopia',
    'process': 'Washed',
    'variety': 'Heirloom',
    'roast_level': 'Light',
    'flavors': ['berries', 'floral', 'citrus'],
    'target_finish_temp': 395,  # Light roast
    'altitude': 2000,  # MASL
}

print("\nGenerating profile for:")
for key, value in demo_specs.items():
    print(f"  {key}: {value}")

# TODO: Encode demo specs
# conditioning = encode_specs(demo_specs)

# TODO: Generate profile
# with torch.no_grad():
#     generated_profile = model.generate(conditioning, start_temp=426, max_steps=800)

# TODO: Plot result
# plt.figure(figsize=(12, 6))
# plt.plot(generated_profile, linewidth=2, color='red')
# plt.xlabel('Time (seconds)')
# plt.ylabel('Temperature (°F)')
# plt.title(f"Generated Profile: {demo_specs['origin']} {demo_specs['process']} - {', '.join(demo_specs['flavors'])}")
# plt.grid(True, alpha=0.3)
# plt.show()

print("\n⚠️  Full demo implementation needed")
print("   See generate_profiles.py for reference")
print("\n💡 Tip: This cell is perfect for live demo during presentation!")
print("   Just update the demo_specs above and run this cell.")
print("="*80)

## 7️⃣ Example Use Cases

Show variety of generated profiles for different beans/flavors.

In [ ]:
# Generate multiple example profiles
print("="*80)
print("EXAMPLE USE CASES")
print("="*80)

examples = [
    {'origin': 'Ethiopia', 'process': 'Washed', 'flavors': ['berries', 'floral'], 'roast': 'Light'},
    {'origin': 'Colombia', 'process': 'Washed', 'flavors': ['chocolate', 'caramel'], 'roast': 'Medium'},
    {'origin': 'Brazil', 'process': 'Natural', 'flavors': ['nuts', 'chocolate'], 'roast': 'Medium'},
    {'origin': 'Kenya', 'process': 'Washed', 'flavors': ['blackcurrant', 'citrus'], 'roast': 'Light'},
]

print("\nExamples to generate:")
for i, ex in enumerate(examples, 1):
    print(f"  {i}. {ex['origin']} {ex['process']} - {', '.join(ex['flavors'])} ({ex['roast']} roast)")

# TODO: Generate and plot all examples
# fig, axes = plt.subplots(2, 2, figsize=(16, 10))
# for idx, (ax, ex) in enumerate(zip(axes.flat, examples)):
#     # Generate profile
#     # profile = generate(ex)
#     # ax.plot(profile)
#     ax.set_title(f"{ex['origin']} {ex['process']} - {', '.join(ex['flavors'])}")

print("\n⚠️  Implementation needed")
print("   These examples showcase model versatility for presentation")
print("="*80)

## 8️⃣ Package Evaluation Results

In [ ]:
# Package all evaluation results
import zipfile
from datetime import datetime
from pathlib import Path

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
package_name = f'roastformer_EVALUATION_{timestamp}.zip'

print("="*80)
print("PACKAGING EVALUATION RESULTS")
print("="*80)

with zipfile.ZipFile(package_name, 'w', zipfile.ZIP_DEFLATED) as zipf:

    # Add visualizations
    if os.path.exists('evaluation_results/real_vs_generated_profiles.png'):
        zipf.write('evaluation_results/real_vs_generated_profiles.png',
                   'real_vs_generated_profiles.png')
        print("✅ Added: real_vs_generated_profiles.png")

    # Add metrics summary
    import json
    metrics_json = json.dumps(metrics, indent=2)
    zipf.writestr('metrics_summary.json', metrics_json)
    print("✅ Added: metrics_summary.json")

    # Create summary
    summary = f"""RoastFormer Evaluation Results
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

═══════════════════════════════════════════════════════════════
EVALUATION METRICS
═══════════════════════════════════════════════════════════════

MAE (Mean Absolute Error): {metrics['mae']:.2f}°F
DTW Distance: {metrics['dtw']:.2f}
Finish Temperature Accuracy: {metrics['finish_temp_accuracy']:.1f}%

Physics Compliance:
  Monotonicity: {metrics['physics_compliance']['monotonicity']:.1f}%
  Bounded RoR: {metrics['physics_compliance']['bounded_ror']:.1f}%
  Smooth Transitions: {metrics['physics_compliance']['smooth_transitions']:.1f}%

═══════════════════════════════════════════════════════════════
FILES INCLUDED
═══════════════════════════════════════════════════════════════

1. real_vs_generated_profiles.png - Visual comparisons
2. metrics_summary.json - Detailed metrics
3. EVALUATION_SUMMARY.txt - This file

═══════════════════════════════════════════════════════════════
NEXT STEPS
═══════════════════════════════════════════════════════════════

1. Use these results to fill EVALUATION_FRAMEWORK.md
2. Include visualizations in presentation
3. Share metrics with Claude for interpretation
4. Discuss limitations and future improvements
"""

    zipf.writestr('EVALUATION_SUMMARY.txt', summary)
    print("✅ Added: EVALUATION_SUMMARY.txt")

print(f"\n📦 Package created: {package_name}")
print(f"   Size: {os.path.getsize(package_name) / 1024 / 1024:.2f} MB")
print("="*80)

In [ ]:
# Download results
from google.colab import files

print("="*80)
print("DOWNLOAD EVALUATION RESULTS")
print("="*80)
print(f"Downloading: {package_name}")
print("="*80)

files.download(package_name)

print("\n✅ Download complete!")

## 🎉 Evaluation Complete!

### What You Have Now:

1. ✅ **Evaluation metrics** - MAE, DTW, physics compliance
2. ✅ **Visual comparisons** - Real vs generated profiles
3. ✅ **Demo-ready notebook** - For live presentation
4. ✅ **Results package** - Everything organized

### Next Steps:

**1. Fill Evaluation Framework:**
- Open `EVALUATION_FRAMEWORK.md` template
- Add actual metrics from this evaluation
- Discuss results and limitations

**2. Prepare Presentation:**
- Use `real_vs_generated_profiles.png` as visual aid
- Practice live demo (this notebook, cell 6)
- Create backup screenshots if demo fails

**3. Critical Analysis:**
- Interpret metrics: What do they mean?
- Discuss flavor ablation impact (if run)
- Identify limitations
- Suggest improvements

---

**Points Secured:** 15/125 (Evaluation) ✅  
**Total Progress:** 85/125 (68%) after training + evaluation  
**Next Milestone:** Critical Analysis + Presentation (20 pts)

---

**💡 Implementation Note:**

This is a **template** showing the structure and flow. To complete:
1. Integrate generation code from `generate_profiles.py`
2. Integrate metrics from `evaluate_transformer.py`
3. Add actual profile plotting
4. Test with real checkpoint

The structure is ready - just need to connect the pieces!

---

**Questions?** Share evaluation results with Claude for interpretation and guidance!